# Pandas support

<div class="alert alert-warning">

**Warning:** pandas support is currently experimental, don't expect everything to work.

</div>

It is convenient to use the Pandas package when dealing with numerical data, so Pint provides PintArray. A PintArray is a Pandas Extension Array, which allows Pandas to recognise the Quantity and store it in Pandas DataFrames and Series.

## Installation


Pandas support is provided by the `pint-pandas` package. To install it use either:
```
python -m pip install pint-pandas
```
Or:
```
conda install -c conda-forge pint-pandas
```

## Basic example

This example will show the simplist way to use pandas with pint and the underlying objects. It's slightly fiddly as you are not reading from a file. A more normal use case is given in Reading a csv.

First some imports

In [ ]:
import pandas as pd 
import pint
import pint_pandas

Next, we create a DataFrame with PintArrays as columns.

In [ ]:
df = pd.DataFrame({
    "torque": pd.Series([1., 2., 2., 3.], dtype="pint[lbf ft]"),
    "angular_velocity": pd.Series([1., 2., 2., 3.], dtype="pint[rpm]"),
})
df

Operations with columns are units aware so behave as we would intuitively expect.

In [ ]:
df['power'] = df['torque'] * df['angular_velocity']
df

We can see the columns' units in the dtypes attribute

In [ ]:
df.dtypes

Each column can be accessed as a Pandas Series

In [ ]:
df.power

Which contains a PintArray

In [ ]:
df.power.values

The PintArray contains a Quantity

In [ ]:
df.power.values.quantity

Pandas Series accessors are provided for most Quantity properties and methods, which will convert the result to a Series where possible.

In [ ]:
df.power.pint.units

In [ ]:
df.power.pint.to("kW").values

## Reading from csv

Reading from files is the far more standard way to use pandas. To facilitate this, DataFrame accessors are provided to make it easy to get to PintArrays. 

In [ ]:
import pandas as pd 
import pint
import pint_pandas
import io

Here's the contents of the csv file.

In [ ]:
test_data = '''ShaftSpeedIndex,rpm,1200,1200,1200,1600,1600,1600,2300,2300,2300
pump,,A,B,C,A,B,C,A,B,C
ShaftSpeed,rpm,1200,1200,1200,1600,1600,1600,2300,2300,2300
FlowRate,m^3 h^-1,8.72,9.28,9.31,11.61,12.78,13.51,18.32,17.90,19.23
DifferentialPressure,kPa,162.03,144.16,136.47,286.86,241.41,204.21,533.17,526.74,440.76
ShaftPower,kW,1.32,1.23,1.18,3.09,2.78,2.50,8.59,8.51,7.61
Efficiency,dimensionless,30.60,31.16,30.70,30.72,31.83,31.81,32.52,31.67,32.05'''

Let's read that into a DataFrame.
Here io.StringIO is used in place of reading a file from disk, whereas a csv file path would typically be used and is shown commented.

In [ ]:
df = pd.read_csv(io.StringIO(test_data), header=[0, 1], index_col = [0,1]).T
# df = pd.read_csv("/path/to/test_data.csv", header=[0, 1])
df

Then use the DataFrame's pint accessor's quantify method to convert the columns from `np.ndarray`s to PintArrays, with units from the bottom column level.

In [ ]:
df.dtypes

In [ ]:
df_ = df.pint.quantify(level=-1)
df_

Let's confirm the units have been parsed correctly

In [ ]:
df_.dtypes

Here the h in m^3 h^-1 has been parsed as the planck constant. Let's change the unit to hours.

In [ ]:
df_['FlowRate'] = pint_pandas.PintArray(df_['FlowRate'].values.quantity.m, dtype = "pint[m^3/hr]")
df_.dtypes

As previously, operations between DataFrame columns are unit aware

In [ ]:
df_.ShaftPower / df_.ShaftSpeed

In [ ]:
df_['ShaftTorque'] = df_.ShaftPower / df_.ShaftSpeed
df_['FluidPower'] = df_['FlowRate'] * df_['DifferentialPressure']
df_

The DataFrame's `pint.dequantify` method then allows us to retrieve the units information as a header row once again.

In [ ]:
df_.pint.dequantify()

This allows for some rather powerful abilities. For example, to change single column units

In [ ]:
df_['FluidPower'] = df_['FluidPower'].pint.to("kW")
df_['FlowRate'] = df_['FlowRate'].pint.to("L/s")
df_['ShaftTorque'] = df_['ShaftTorque'].pint.to("N m")
df_.pint.dequantify()

The units are harder to read than they need be, so lets change pints default format for displaying units.

In [ ]:
pint_pandas.PintType.ureg.default_format = "~P"
df_.pint.dequantify()

or the entire table's units

In [ ]:
df_.pint.to_base_units().pint.dequantify()

## Plotting
Pint's matplotlib support allows columns with the same dimensionality to be plotted.

In [ ]:
pint_pandas.PintType.ureg.setup_matplotlib()
ax = df_[['ShaftPower', 'FluidPower']].unstack("pump").plot()

In [ ]:
ax.yaxis.units

Note that indexes cannot store PintArrays, so don't contain unit information

In [ ]:
print(ax.xaxis.units)

## Advanced example
This example shows alternative ways to use pint with pandas and other features.

Start with the same imports.

In [ ]:
import pandas as pd 
import pint
import pint_pandas

We'll be use a shorthand for PintArray

In [ ]:
PA_ = pint_pandas.PintArray

And set up a unit registry and quantity shorthand.

In [ ]:
ureg = pint.UnitRegistry()
Q_ = ureg.Quantity

Operations between PintArrays of different unit registry will not work. We can change the unit registry that will be used in creating new PintArrays to prevent this issue.

In [ ]:
pint_pandas.PintType.ureg = ureg

These are the possible ways to create a PintArray.

Note that pint[unit] must be used for the Series constuctor, whereas the PintArray constructor allows the unit string or object.

In [ ]:
df = pd.DataFrame({
        "length" : pd.Series([1.,2.], dtype="pint[m]"),
        "width" : PA_([2.,3.], dtype="pint[m]"),
        "distance" : PA_([2.,3.], dtype="m"),
        "height" : PA_([2.,3.], dtype=ureg.m),
        "depth" : PA_.from_1darray_quantity(Q_([2,3],ureg.m)),
    })
df

In [ ]:
df.length.values.units